# Kl 25 FIR filter experiments
In this notebook experiments with the KL25 Cortex m0+ CPU and builtin 12 bit DAC and 16 ADC is described.

## KL25 ADC
The ADC is a 16 bit Sucessive Approximation converter that can be setup in 8/10/12/14/16 bit mode. Conversion clock runs of a prescaled bus clock, derived from the 48MHz CPU clock

### Features (from user manual )
Features of the ADC module include:
 * Linear successive approximation algorithm with up to 16-bit resolution
 * Up to four pairs of differential and 24 single-ended external analog inputs
 * Output modes:
 * differential 16-bit, 13-bit, 11-bit, and 9-bit modes
 * single-ended 16-bit, 12-bit, 10-bit, and 8-bit modes
 * Output format in 2's complement 16-bit sign extended for differential modes
 * Output in right-justified unsigned format for single-ended
 * Single or continuous conversion, that is, automatic return to idle after single  conversion

## System design 
The digital to analog converter **DAC** is a 12 bit converter capable of outputting 0-3.3V.

The **ADC** will be initialised in 12 bit mode, to provide identical input and output resolution

### Functional blocks
![BLOCK DIAGRAM](./img/FIR_KL25.png)


### Conversion & processing times

 *  \$t_{ADC} $  = 3,6$\mu s$ 

 *  \$t_{DAC} $  = 1.2$\mu s$

# A boilerplate FIR filter
Template code from [http://t-filter.engineerjs.com/]
with a bandstop filter attenuating 250-600Hz is shown below

```
    /*
    FIR filter designed with
     http://t-filter.appspot.com
    sampling frequency: 6000 Hz
    fixed point precision: 16 bits

    * 0 Hz - 100 Hz
      gain = 1
      desired ripple = 5 dB
      actual ripple = n/a

    * 250 Hz - 600 Hz
      gain = 0
      desired attenuation = -40 dB
      actual attenuation = n/a

    * 750 Hz - 3000 Hz
      gain = 1
      desired ripple = 5 dB
      actual ripple = n/a
    */

    #define SAMPLEFILTER_TAP_NUM 43

    typedef struct {
      int history[SAMPLEFILTER_TAP_NUM];
      unsigned int last_index;
    } SampleFilter;

    void SampleFilter_init(SampleFilter* f);
    void SampleFilter_put(SampleFilter* f, int input);
    int SampleFilter_get(SampleFilter* f);

    #endif
```

The FIR loop is not optimized for speed, and implemented with plain Multiply Accumulate **MAC** operations

```
void SampleFilter_init(SampleFilter* f) {
  int i;
  for(i = 0; i < SAMPLEFILTER_TAP_NUM; ++i)
    f->history[i] = 0;
  f->last_index = 0;
}

void SampleFilter_put(SampleFilter* f, int input) {
  f->history[f->last_index++] = input;
  if(f->last_index == SAMPLEFILTER_TAP_NUM)
    f->last_index = 0;
}

int SampleFilter_get(SampleFilter* f) {
  long long acc = 0;
  int index = f->last_index, i;
  for(i = 0; i < SAMPLEFILTER_TAP_NUM; ++i) {
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[i];
  };
  return acc >> 16;
}
```




### FIR execution time 
  *  \$t_{Filter 43} $  = 96$\mu s$

In [4]:
t_filter_43 = 96e-6
t_filter_43

9.6e-05

### Optimize with loop unrolling
The 43 taps can be unrolled, to avoid branches in the code, and do linear code execution, that may speed up execution

In [ ]:
int SampleFilter_get(SampleFilter* f) {
  long long acc = 0;
  int index = f->last_index;
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[0];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[1];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[2];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[3];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[4];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[5];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[6];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[7];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[8];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[9];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[10];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[11];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[12];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[13];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[14];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[15];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[16];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[17];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[18];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[19];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[20];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[21];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[22];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[23];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[24];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[25];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[26];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[27];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[28];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[29];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[30];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[31];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[32];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[33];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[34];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[35];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[36];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[37];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[38];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[39];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[40];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[41];
    index = index != 0 ? index-1 : SAMPLEFILTER_TAP_NUM-1;
    acc += (long long)f->history[index] * filter_taps[42];
  return acc >> 16;
}

### FIR execution time wit unrolled filter loop
  *  \$t_{Filter 43 unrolled} $  = 105$\mu s$
  
That actually took **longer** time that the looped version